## EDA Notebook
- V1 on 9/17/2025 8pm PT

In [7]:
!pip install --quiet gdown pandas

#### Download 'reported.tsv'

In [14]:
file_id = "1nBFjtz1QsrYX96hKYXl1BMhnS-0lJrFK"  # reported.tsv
download_url = f"https://drive.google.com/uc?id={file_id}"
!gdown {download_url} -O reported.tsv

Downloading...
From: https://drive.google.com/uc?id=1nBFjtz1QsrYX96hKYXl1BMhnS-0lJrFK
To: /home/sagemaker-user/reported.tsv
100%|██████████████████████████████████████| 1.39M/1.39M [00:00<00:00, 36.6MB/s]


In [16]:
reported_df = pd.read_csv("reported.tsv", sep="\t", on_bad_lines="skip")
print(f"reported_df.shape: {reported_df.shape}")
reported_df.head(2)

reported_df.shape: (9150, 4)


,sentence_id,sentence,locale,reason
0,0e047d99c6c75b076aa34cdd37238670b0a9340fb78737...,It should not be confused with the Novodevichy...,en,grammar-or-spelling
1,0e047d99c6c75b076aa34cdd37238670b0a9340fb78737...,It should not be confused with the Novodevichy...,en,offensive-language


#### Download 'train.tsv'

In [17]:
file_id = "1W3gdWpXUnjJn_nwTpvlWwLLAfF1YPwos"  # train.tsv
download_url = f"https://drive.google.com/uc?id={file_id}"
!gdown {download_url} -O train.tsv

Downloading...
From (original): https://drive.google.com/uc?id=1W3gdWpXUnjJn_nwTpvlWwLLAfF1YPwos
From (redirected): https://drive.google.com/uc?id=1W3gdWpXUnjJn_nwTpvlWwLLAfF1YPwos&confirm=t&uuid=e2ad4626-44d8-46c6-a400-c7562e00ff23
To: /home/sagemaker-user/train.tsv
100%|█████████████████████████████████████████| 376M/376M [00:02<00:00, 154MB/s]


In [18]:
train_df = pd.read_csv("train.tsv", sep="\t")
print(f"train_df.shape: {train_df.shape}")
train_df.head(2)

/tmp/ipykernel_152/706301808.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("train.tsv", sep="\t")


train_df.shape: (1138422, 13)


,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,d41335955114ab117fd234b8e17a8ef833dddcf5c384f0...,common_voice_en_23679703.mp3,70ada06f3bc0b24fea1ad1bdb6d8245eed274e766642d7...,He directed four episodes of the series from s...,NaN,2,0,NaN,NaN,NaN,NaN,en,NaN
1,d41335955114ab117fd234b8e17a8ef833dddcf5c384f0...,common_voice_en_23679704.mp3,70a88237ebc082bb8817eec04735dbed3dc9a092e3a68a...,He was Emeritus Vilas Research Professor at th...,NaN,2,0,NaN,NaN,NaN,NaN,en,NaN


In [37]:
train_df["locale"].value_counts()

locale
en    1138422
Name: count, dtype: int64

#### Filter to only rows having age

In [19]:
train_with_non_null_age = train_df[train_df["age"].notna()]
print(f"Removing nulls reduces to {train_with_non_null_age.shape[0]}/{train_df.shape[0]} rows")

Removing nulls reduces to 811179/1138422 rows


#### Count and percentage of age buckets

In [20]:
train_with_non_null_age["age"].value_counts()

age
twenties     311490
thirties     164119
fourties     114433
teens         77153
fifties       73417
sixties       62614
seventies      6578
eighties       1179
nineties        196
Name: count, dtype: int64

In [21]:
train_with_non_null_age["age"].value_counts(normalize=True)*100

age
twenties     38.399663
thirties     20.232156
fourties     14.106997
teens         9.511218
fifties       9.050653
sixties       7.718888
seventies     0.810918
eighties      0.145344
nineties      0.024162
Name: proportion, dtype: float64

#### Count of age + gender

In [23]:
# Age + Gender crosstab
age_order = [
    "teens", "twenties", "thirties", "fourties",
    "fifties", "sixties", "seventies", "eighties", "nineties"
]
ct = pd.crosstab(
    index=train_with_non_null_age["age"],
    columns=train_with_non_null_age["gender"],
    normalize="index"
) * 100
ct.reindex(age_order)

gender,do_not_wish_to_say,female_feminine,male_masculine,non-binary,transgender
age,,,,,
teens,0.000000,38.027127,61.959280,0.000000,0.013594
twenties,0.010671,30.257226,69.717432,0.013672,0.001000
thirties,0.000000,19.495558,80.504442,0.000000,0.000000
fourties,0.000000,21.304290,78.693059,0.002651,0.000000
fifties,0.000000,27.055601,72.944399,0.000000,0.000000
sixties,0.000000,54.730974,45.269026,0.000000,0.000000
seventies,0.000000,39.935711,60.064289,0.000000,0.000000
eighties,0.000000,20.613288,79.386712,0.000000,0.000000
nineties,0.000000,0.000000,100.000000,0.000000,0.000000


In [31]:
# pd.set_option("display.max_rows", None)
pd.reset_option("display.max_rows")

#### Count of accent

In [36]:
# accent_value_counts_df = train_with_non_null_age["accents"].value_counts()
# print(accent_value_counts_df)

# accent_value_counts_df = train_with_non_null_age["accents"].value_counts().reset_index()
# accent_value_counts_df.columns = ["accent", "count"]
# print(accent_value_counts_df.to_string(index=False))

accent_value_counts_df["accent_short"] = accent_value_counts_df["accent"].str.slice(0, 40) + "…"
print(f"accent_value_counts_df.shape: {accent_value_counts_df.shape}")
print(accent_value_counts_df[["accent_short", "count"]].to_string(index=False))

accent_value_counts_df.shape: (344, 3)
                             accent_short  count
                   United States English… 260640
                         England English…  78239
India and South Asia (India, Pakistan, S…  77545
                        Canadian English…  54193
       Non native speaker,German English…  51696
                      Australian English…  33175
Southern African (South Africa, Zimbabwe…  21063
                        Scottish English…   8958
                          Northern Irish…   6209
                     New Zealand English…   5986
                           Irish English…   5755
                                Filipino…   4723
   United States English,England English…   2804
                     Singaporean English…   2769
                       Hong Kong English…   2759
England English,Liverpool English,Lancas…   2574
     England English,New Zealand English…   2565
United States English,Midwestern,Low,Dem…   1364
United States English,Transatl

#### Count of accent

In [25]:
# Age + Accent crosstab
age_order = [
    "teens", "twenties", "thirties", "fourties",
    "fifties", "sixties", "seventies", "eighties", "nineties"
]
ct = pd.crosstab(
    index=train_with_non_null_age["age"],
    columns=train_with_non_null_age["accents"],
    normalize="index"
)*100
ct.reindex(age_order)

accents,2nd Language,"2nd language, South African (first language Afrikaans)",A variety of Texan English with some German influence that has undergone the cot-caught merger,A'lo,African Accent,Afrikaans English,Australian English,"Australian English,Canadian English","Australian English,Educated Australian Accent","Australian English,England English",...,nigeria english,nigerian accent,northen united states,"not native English, originally French native language",polish,polish accent,serbian,strong Latvian accent,"very slight Russian accent,Standard American English,Boston influence","Немного русский акцент произношения, съедание слов"
age,,,,,,,,,,,,,,,,,,,,,
teens,0.043923,0.000000,0.021961,0.000000,0.000000,0.000000,16.813920,0.011825,0.000000,0.032097,...,0.000000,0.000000,0.000000,0.000000,0.483149,0.000000,0.008447,0.000000,0.00000,0.008447
twenties,0.000000,0.000000,0.000000,0.000000,0.000000,0.001712,4.665417,0.000000,0.000000,0.000000,...,0.068911,0.079184,0.000000,0.000000,0.007704,0.003424,0.000000,0.000000,0.00214,0.000000
thirties,0.000000,0.000000,0.000000,0.000000,0.004259,0.000000,3.494364,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.006388,0.003549,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
fourties,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.599984,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.019349,0.000000,0.000000,0.000000,0.00000,0.000000
fifties,0.000000,0.003048,0.000000,0.006096,0.000000,0.000000,2.029838,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080767,0.00000,0.000000
sixties,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.968649,0.000000,0.052169,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
seventies,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.971678,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
eighties,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.355705,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
nineties,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
